In [189]:
# Import libaries
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import creds
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

In [190]:
# Reading in song dataset
df = pd.read_csv('drake_songs_dataset.csv')
df

,track_uri,track_name,album_name,duration_ms,danceability,energy,key,loudness,speechiness,acousticness,...,liveness,valence,tempo,mode,type,id,uri,track_href,analysis_url,time_signature
0,spotify:track:67nepsnrcZkowTxMWigSbb,MELTDOWN (feat. Drake),UTOPIA,246134,0.557,0.774,7,-5.275,0.3510,0.012000,...,0.3960,0.397,111.975,0,audio_features,67nepsnrcZkowTxMWigSbb,spotify:track:67nepsnrcZkowTxMWigSbb,https://api.spotify.com/v1/tracks/67nepsnrcZko...,https://api.spotify.com/v1/audio-analysis/67ne...,4
1,spotify:track:6DCZcSspjsKoFjzjrWoCdn,God’s Plan,Scorpion,198973,0.754,0.449,7,-9.211,0.1090,0.033200,...,0.5520,0.357,77.169,1,audio_features,6DCZcSspjsKoFjzjrWoCdn,spotify:track:6DCZcSspjsKoFjzjrWoCdn,https://api.spotify.com/v1/tracks/6DCZcSspjsKo...,https://api.spotify.com/v1/audio-analysis/6DCZ...,4
2,spotify:track:3F5CgOj3wFlRv51JsHbxhe,Jimmy Cooks (feat. 21 Savage),"Honestly, Nevermind",218365,0.529,0.673,0,-4.711,0.1750,0.000307,...,0.0930,0.366,165.921,1,audio_features,3F5CgOj3wFlRv51JsHbxhe,spotify:track:3F5CgOj3wFlRv51JsHbxhe,https://api.spotify.com/v1/tracks/3F5CgOj3wFlR...,https://api.spotify.com/v1/audio-analysis/3F5C...,4
3,spotify:track:2G7V7zsVDxg1yRsu7Ew9RJ,In My Feelings,Scorpion,217925,0.835,0.626,1,-5.833,0.1250,0.058900,...,0.3960,0.350,91.030,1,audio_features,2G7V7zsVDxg1yRsu7Ew9RJ,spotify:track:2G7V7zsVDxg1yRsu7Ew9RJ,https://api.spotify.com/v1/tracks/2G7V7zsVDxg1...,https://api.spotify.com/v1/audio-analysis/2G7V...,4
4,spotify:track:7aRCf5cLOFN1U7kvtChY1G,Search & Rescue,Search & Rescue,272113,0.817,0.440,10,-8.482,0.0734,0.060300,...,0.3300,0.544,142.024,0,audio_features,7aRCf5cLOFN1U7kvtChY1G,spotify:track:7aRCf5cLOFN1U7kvtChY1G,https://api.spotify.com/v1/tracks/7aRCf5cLOFN1...,https://api.spotify.com/v1/audio-analysis/7aRC...,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,spotify:track:3ZF3TivVsS8MZ7wD0ZT3ZA,Aston Martin Music - Edit,Teflon Don,270627,0.577,0.569,10,-5.579,0.2230,0.012400,...,0.0535,0.383,171.825,0,audio_features,3ZF3TivVsS8MZ7wD0ZT3ZA,spotify:track:3ZF3TivVsS8MZ7wD0ZT3ZA,https://api.spotify.com/v1/tracks/3ZF3TivVsS8M...,https://api.spotify.com/v1/audio-analysis/3ZF3...,4
421,spotify:track:5JEnw81UmKRf1LRvHJqwR8,WASTING TIME (FEAT. DRAKE & THE NEPTUNES) (CHO...,WASTELAND - CHOPPED NOT SLOPPED,364762,0.629,0.687,4,-3.709,0.0952,0.198000,...,0.2920,0.327,150.072,0,audio_features,5JEnw81UmKRf1LRvHJqwR8,spotify:track:5JEnw81UmKRf1LRvHJqwR8,https://api.spotify.com/v1/tracks/5JEnw81UmKRf...,https://api.spotify.com/v1/audio-analysis/5JEn...,4
422,spotify:track:0zvAHtoa1v8oE9eRGVnwjV,Moment 4 Life (feat. Drake),Queen Radio: Volume 1,279307,0.500,0.882,11,-3.846,0.3790,0.401000,...,0.2840,0.401,130.483,1,audio_features,0zvAHtoa1v8oE9eRGVnwjV,spotify:track:0zvAHtoa1v8oE9eRGVnwjV,https://api.spotify.com/v1/tracks/0zvAHtoa1v8o...,https://api.spotify.com/v1/audio-analysis/0zvA...,5
423,spotify:track:5vslB8gLeWNYUrUPepRCi0,P Power,slow motion easy rhymes,193347,0.828,0.591,1,-6.217,0.1090,0.019900,...,0.3610,0.202,119.251,1,audio_features,5vslB8gLeWNYUrUPepRCi0,spotify:track:5vslB8gLeWNYUrUPepRCi0,https://api.spotify.com/v1/tracks/5vslB8gLeWNY...,https://api.spotify.com/v1/audio-analysis/5vsl...,4


In [191]:
# Checking for null values
df.isna().sum()

track_uri           0
track_name          0
album_name          0
duration_ms         0
danceability        0
energy              0
key                 0
loudness            0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
mode                0
type                0
id                  0
uri                 0
track_href          0
analysis_url        0
time_signature      0
dtype: int64

In [192]:
# Get features we want to include in the cosine similarity calculation
features = [
        'danceability', 'energy', 'key', 'loudness',
        'speechiness', 'acousticness', 'instrumentalness', 
        'liveness', 'valence', 'tempo'
    ]

In [193]:
df[features]

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.557,0.774,7,-5.275,0.3510,0.012000,0.000000,0.3960,0.397,111.975
1,0.754,0.449,7,-9.211,0.1090,0.033200,0.000083,0.5520,0.357,77.169
2,0.529,0.673,0,-4.711,0.1750,0.000307,0.000002,0.0930,0.366,165.921
3,0.835,0.626,1,-5.833,0.1250,0.058900,0.000060,0.3960,0.350,91.030
4,0.817,0.440,10,-8.482,0.0734,0.060300,0.000001,0.3300,0.544,142.024
...,...,...,...,...,...,...,...,...,...,...
420,0.577,0.569,10,-5.579,0.2230,0.012400,0.000000,0.0535,0.383,171.825
421,0.629,0.687,4,-3.709,0.0952,0.198000,0.000011,0.2920,0.327,150.072
422,0.500,0.882,11,-3.846,0.3790,0.401000,0.000000,0.2840,0.401,130.483
423,0.828,0.591,1,-6.217,0.1090,0.019900,0.000323,0.3610,0.202,119.251


In [194]:
# Function to get information about the user's track, returns track info (dictionary) or None if not a valid song
def get_user_track():
    # Get user track
    track = input('Enter track name: ')
    # Make API call to verify if it is a spotify song
    # Initialize the Spotify API client with client credentials
    sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=creds.CLIENT_ID, client_secret=creds.CLIENT_SECRET))

    # Make API call to search for the user's song made by Drake and extract the audio features of the song if a valid song
    search_results = sp.search(q='track:' + track + ' artist:Drake', type='track', limit=1)

    # Check if song is an actual track in Spotify
    if search_results['tracks']['items']:
        return search_results
    return None


In [195]:
# Function that calculates cosine similarity and makes recommendations
def cosine_recs(drake_dataset, features, n_recs):
    # Get the user's track
    user_track = get_user_track()
    # Check if valid track
    if user_track is not None:
        # Check if track is in the drake song dataset
        if user_track['tracks']['items'][0]['name'] in drake_dataset['track_name'].values:
            # Start recommendation system
            
            # Create an indices series to get the index of a track given its name
            indices = pd.Series(drake_dataset.index, index = drake_dataset['track_name'])
            user_idx = indices[user_track['tracks']['items'][0]['name']]

            # Scale data
            scaler =  StandardScaler()
            scaled_df = scaler.fit_transform(drake_dataset[features])

            # Calculate cosine similarity matrix
            cos_matrix = cosine_similarity(scaled_df, scaled_df)

            # Create a series with the similarity scores in descending order
            scores = pd.Series(cos_matrix[user_idx]).sort_values(ascending = False)

            # Get indexes of top N recommendations
            rec_indexes = list(scores.iloc[1:(n_recs + 1)].index)

            return df.iloc[rec_indexes]
            
    return None
    

In [198]:
cosine_recs(df, features, 5)

,track_uri,track_name,album_name,duration_ms,danceability,energy,key,loudness,speechiness,acousticness,...,liveness,valence,tempo,mode,type,id,uri,track_href,analysis_url,time_signature
293,spotify:track:2jTujnt0y344ai1rNOywgr,KMT,More Life,162638,0.875,0.600,1,-6.818,0.0435,0.01890,...,0.1320,0.0906,119.987,1,audio_features,2jTujnt0y344ai1rNOywgr,spotify:track:2jTujnt0y344ai1rNOywgr,https://api.spotify.com/v1/tracks/2jTujnt0y344...,https://api.spotify.com/v1/audio-analysis/2jTu...,4
50,spotify:track:1DMYEiuAgz1OKvANXiNFrN,Recognize (feat. Drake),PARTYNEXTDOOR TWO,311197,0.644,0.541,2,-5.801,0.0426,0.01020,...,0.1020,0.1650,119.513,1,audio_features,1DMYEiuAgz1OKvANXiNFrN,spotify:track:1DMYEiuAgz1OKvANXiNFrN,https://api.spotify.com/v1/tracks/1DMYEiuAgz1O...,https://api.spotify.com/v1/audio-analysis/1DMY...,4
144,spotify:track:6UjfByV1lDLW0SOVQA4NAi,Gyalchester,More Life,189230,0.877,0.391,1,-8.196,0.0630,0.03170,...,0.0945,0.1220,131.018,1,audio_features,6UjfByV1lDLW0SOVQA4NAi,spotify:track:6UjfByV1lDLW0SOVQA4NAi,https://api.spotify.com/v1/tracks/6UjfByV1lDLW...,https://api.spotify.com/v1/audio-analysis/6Ujf...,4
216,spotify:track:0GR7iJLhj80KD5LkA14ZRn,Digital Dash,What A Time To Be Alive,231231,0.881,0.546,2,-7.566,0.1200,0.00296,...,0.2260,0.2760,140.032,1,audio_features,0GR7iJLhj80KD5LkA14ZRn,spotify:track:0GR7iJLhj80KD5LkA14ZRn,https://api.spotify.com/v1/tracks/0GR7iJLhj80K...,https://api.spotify.com/v1/audio-analysis/0GR7...,4
112,spotify:track:4c2xt1trwYZpMqPWY35Xi9,Jaded,Scorpion,262453,0.764,0.271,0,-5.697,0.0414,0.03700,...,0.1750,0.0371,116.019,1,audio_features,4c2xt1trwYZpMqPWY35Xi9,spotify:track:4c2xt1trwYZpMqPWY35Xi9,https://api.spotify.com/v1/tracks/4c2xt1trwYZp...,https://api.spotify.com/v1/audio-analysis/4c2x...,4
